# 串口数据帧解析与校验
本代码用于解析形如“AA BB ... CC DD”的串口数据，每帧10字节，校验并显示每帧的校验结果。

In [2]:
# 输入串口数据（十六进制字符串，空格分隔）
data_str = "AA BB 00 00 14 C7 FF 00 CC DD AA BB 00 00 17 3F FF 00 CC DD AA BB 00 00 17 00 FF 00 CC DD AA BB 00 00 17 13 FF 00 CC DD AA BB 00 00 17 3F 1F 00 CC DD "

# 转换为字节列表
data_bytes = [int(x, 16) for x in data_str.strip().split()]

FRAME_LEN = 10
HEAD = [0xAA, 0xBB]
TAIL = [0xCC, 0xDD]

def parse_frames(data_bytes):
    frames = []
    i = 0
    while i <= len(data_bytes) - FRAME_LEN:
        if data_bytes[i:i+2] == HEAD and data_bytes[i+8:i+10] == TAIL:
            frame = data_bytes[i:i+FRAME_LEN]
            frames.append(frame)
            i += FRAME_LEN
        else:
            i += 1
    return frames

def check_frame(frame):
    # 第3-7字节（索引2-6）异或校验，结果应等于第9字节（索引8）
    xor_val = 0
    for b in frame[2:7]:
        xor_val ^= b
    return xor_val == frame[7], xor_val, frame[7]

frames = parse_frames(data_bytes)

for idx, frame in enumerate(frames):
    ok, calc_xor, frame_xor = check_frame(frame)
    print(f"帧{idx+1}: {['%02X'%b for b in frame]}")
    print(f"  校验{'正确' if ok else '错误'}，计算值: {calc_xor:02X}，帧内值: {frame_xor:02X}\n")

帧1: ['AA', 'BB', '00', '00', '14', 'C7', 'FF', '00', 'CC', 'DD']
  校验错误，计算值: 2C，帧内值: 00

帧2: ['AA', 'BB', '00', '00', '17', '3F', 'FF', '00', 'CC', 'DD']
  校验错误，计算值: D7，帧内值: 00

帧3: ['AA', 'BB', '00', '00', '17', '00', 'FF', '00', 'CC', 'DD']
  校验错误，计算值: E8，帧内值: 00

帧4: ['AA', 'BB', '00', '00', '17', '13', 'FF', '00', 'CC', 'DD']
  校验错误，计算值: FB，帧内值: 00

帧5: ['AA', 'BB', '00', '00', '17', '3F', '1F', '00', 'CC', 'DD']
  校验错误，计算值: 37，帧内值: 00

